# 🔗 DataFrame Joins: Inner, Left, Right & Full

**Time to complete:** 40 minutes  
**Difficulty:** Intermediate  
**Prerequisites:** DataFrame basics, aggregations

---

## 🎯 Learning Objectives

By the end of this notebook, you will master:
- ✅ **Inner Join** - Only matching rows
- ✅ **Left Join** - All left rows, matching right rows
- ✅ **Right Join** - All right rows, matching left rows
- ✅ **Full Outer Join** - All rows from both sides
- ✅ **Join conditions** - Equality and non-equality joins
- ✅ **Multiple table joins** - Joining more than 2 DataFrames
- ✅ **Performance optimization** - Efficient join strategies

**Joins are fundamental to relational data processing!**

---

## 🔍 Understanding DataFrame Joins

**Joins combine data from multiple DataFrames based on common columns or conditions.**

### Join Types:
- **Inner Join**: Only rows with matching keys in both DataFrames
- **Left Join**: All rows from left DataFrame, matching rows from right
- **Right Join**: All rows from right DataFrame, matching rows from left
- **Full Outer Join**: All rows from both DataFrames
- **Cross Join**: Cartesian product of all rows
- **Anti Join**: Rows that don't have matches

### Join Performance:
- **Broadcast Join**: Small table fits in memory
- **Shuffle Hash Join**: Large tables, hash-based
- **Sort-Merge Join**: Large sorted tables
- **Nested Loop Join**: Fallback, usually slow

**Joins are expensive operations - they require data shuffling!**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, coalesce
import pyspark.sql.functions as F

spark = SparkSession.builder \
    .appName("DataFrame_Joins") \
    .master("local[*]") \
    .getOrCreate()

print(f"✅ Spark ready - Version: {spark.version}")

# Create employees DataFrame
employees_data = [
    (1, "Alice", "Engineering", 75000),
    (2, "Bob", "Sales", 65000),
    (3, "Charlie", "Engineering", 85000),
    (4, "Diana", "HR", 55000),
    (5, "Eve", "Sales", 70000),
    (6, "Frank", "Engineering", 80000)
]

employees_df = spark.createDataFrame(employees_data, 
    ["emp_id", "name", "department", "salary"])

# Create departments DataFrame
departments_data = [
    ("Engineering", "Building software", "New York"),
    ("Sales", "Selling products", "Chicago"),
    ("HR", "Managing people", "Boston"),
    ("Marketing", "Brand promotion", "Los Angeles"),
    ("Finance", "Financial management", "New York")
]

departments_df = spark.createDataFrame(departments_data, 
    ["dept_name", "description", "location"])

# Create projects DataFrame
projects_data = [
    (101, "Website Redesign", "Engineering", 3),
    (102, "Sales CRM", "Sales", 2),
    (103, "Talent Acquisition", "HR", 1),
    (104, "Brand Campaign", "Marketing", 4),
    (105, "Budget Planning", "Finance", 2)
]

projects_df = spark.createDataFrame(projects_data, 
    ["project_id", "project_name", "department", "team_size"])

print("📊 Sample DataFrames:")
print("\nEmployees:")
employees_df.show()

print("Departments:")
departments_df.show()

print("Projects:")
projects_df.show()

## 🎯 Inner Join

**Inner Join returns only rows that have matching values in both DataFrames.**

### Characteristics:
- ✅ **Only matching rows** from both sides
- ✅ **Most common join type**
- ✅ **Reduces data size** - only intersections
- ✅ **Symmetric operation**

### Use Cases:
- Finding related data across tables
- Data validation and integrity checks
- Creating combined views for analysis

In [ ]:
# Inner join examples
print("🎯 INNER JOIN EXAMPLES")
print("=" * 50)

# Join employees with departments
emp_dept_inner = employees_df.join(
    departments_df,
    employees_df["department"] == departments_df["dept_name"],
    "inner"
)

print("Inner join - Employees with departments:")
emp_dept_inner.select(
    "name", "department", "salary", "description", "location"
).show()

print(f"\nOriginal employees: {employees_df.count()} rows")
print(f"Original departments: {departments_df.count()} rows")
print(f"Inner join result: {emp_dept_inner.count()} rows")

# Join employees with projects (both have department column)
emp_projects_inner = employees_df.join(
    projects_df,
    "department",  # Join on common column name
    "inner"
)

print("\nInner join - Employees with projects:")
emp_projects_inner.select(
    "name", "department", "project_name", "team_size"
).show()

print(f"\nProjects inner join result: {emp_projects_inner.count()} rows")

### Inner Join Analysis

Notice how:
- **Frank** (Finance) is excluded because Finance department doesn't exist in departments_df
- **Marketing & Finance** departments are excluded because they have no employees
- Only **matching pairs** are returned

**Inner joins are great for finding complete, related data.**

## ⬅️ Left Join (Left Outer Join)

**Left Join returns all rows from the left DataFrame and matching rows from the right DataFrame.**

### Characteristics:
- ✅ **All rows from left DataFrame**
- ✅ **Matching rows from right DataFrame**
- ✅ **NULL values** for non-matching right rows
- ✅ **Preserves left data integrity**

### Use Cases:
- Preserving all customer/order data
- Finding missing relationships
- Left-side dominant analysis

In [ ]:
# Left join examples
print("⬅️ LEFT JOIN EXAMPLES")
print("=" * 50)

# Left join employees with departments
emp_dept_left = employees_df.join(
    departments_df,
    employees_df["department"] == departments_df["dept_name"],
    "left"
)

print("Left join - All employees, matching departments:")
emp_dept_left.select(
    "name", "department", "salary", "description", "location"
).show()

print(f"\nOriginal employees: {employees_df.count()} rows")
print(f"Left join result: {emp_dept_left.count()} rows")
print("(Same as original - all employees included)")

# Find employees without department info
orphaned_employees = emp_dept_left.filter(
    col("dept_name").isNull()
)

print("\nEmployees without department info:")
orphaned_employees.select("name", "department", "dept_name").show()

# Left join projects with departments
proj_dept_left = projects_df.join(
    departments_df,
    projects_df["department"] == departments_df["dept_name"],
    "left"
)

print("\nLeft join - All projects, matching departments:")
proj_dept_left.select(
    "project_name", "department", "description", "location"
).show()

print("\nProjects without department info:")
proj_dept_left.filter(col("dept_name").isNull()).select(
    "project_name", "department"
).show()

### Left Join Analysis

Key observations:
- **All employees included** (6 rows, same as original)
- **Frank** (Finance) has NULL values for department info
- **Marketing & Finance** departments are excluded (no employees)
- **Preserves left DataFrame integrity**

**Left joins are perfect for 'what's missing' analysis.**

## ➡️ Right Join (Right Outer Join)

**Right Join returns all rows from the right DataFrame and matching rows from the left DataFrame.**

### Characteristics:
- ✅ **All rows from right DataFrame**
- ✅ **Matching rows from left DataFrame**
- ✅ **NULL values** for non-matching left rows
- ✅ **Symmetric to left join** (just swap DataFrames)

### Use Cases:
- Preserving all reference/lookup data
- Finding unused resources
- Right-side dominant analysis

In [ ]:
# Right join examples
print("➡️ RIGHT JOIN EXAMPLES")
print("=" * 50)

# Right join employees with departments
emp_dept_right = employees_df.join(
    departments_df,
    employees_df["department"] == departments_df["dept_name"],
    "right"
)

print("Right join - All departments, matching employees:")
emp_dept_right.select(
    "name", "department", "dept_name", "description", "location"
).orderBy("dept_name").show()

print(f"\nOriginal departments: {departments_df.count()} rows")
print(f"Right join result: {emp_dept_right.count()} rows")

# Find departments without employees
empty_departments = emp_dept_right.filter(
    col("name").isNull()
)

print("\nDepartments without employees:")
empty_departments.select("dept_name", "description").show()

# Right join projects with employees
proj_emp_right = projects_df.join(
    employees_df,
    projects_df["department"] == employees_df["department"],
    "right"
)

print("\nRight join - All employees, matching projects:")
proj_emp_right.select(
    "name", "department", "project_name", "team_size"
).show()

print("\nEmployees without projects:")
proj_emp_right.filter(col("project_name").isNull()).select(
    "name", "department"
).show()

### Right Join Analysis

Key observations:
- **All departments included** (5 rows, same as original)
- **Marketing & Finance** departments have NULL employee values
- **All employees with projects** are shown
- **Preserves right DataFrame integrity**

**Right joins help find 'unused capacity' or 'missing assignments'.**

## 🔄 Full Outer Join

**Full Outer Join returns all rows from both DataFrames, with NULLs where there's no match.**

### Characteristics:
- ✅ **All rows from both DataFrames**
- ✅ **NULL values** for non-matching rows
- ✅ **Complete picture** of relationships
- ✅ **Largest result set** of all join types

### Use Cases:
- Complete data reconciliation
- Finding all relationships and gaps
- Data quality assessment
- Comprehensive reporting

In [ ]:
# Full outer join examples
print("🔄 FULL OUTER JOIN EXAMPLES")
print("=" * 50)

# Full outer join employees with departments
emp_dept_full = employees_df.join(
    departments_df,
    employees_df["department"] == departments_df["dept_name"],
    "full"
)

print("Full outer join - All employees and all departments:")
emp_dept_full.select(
    "name", "department", "dept_name", "description", "location"
).orderBy(
    coalesce(col("department"), col("dept_name")).asc()
).show()

print(f"\nOriginal employees: {employees_df.count()} rows")
print(f"Original departments: {departments_df.count()} rows")
print(f"Full outer join result: {emp_dept_full.count()} rows")
print(f"Expected: {employees_df.count() + departments_df.count()} rows")

# Analyze join results
matched = emp_dept_full.filter(col("name").isNotNull() & col("dept_name").isNotNull())
only_employees = emp_dept_full.filter(col("name").isNotNull() & col("dept_name").isNull())
only_departments = emp_dept_full.filter(col("name").isNull() & col("dept_name").isNotNull())

print(f"\nJoin analysis:")
print(f"- Matched records: {matched.count()}")
print(f"- Only in employees: {only_employees.count()}")
print(f"- Only in departments: {only_departments.count()}")

# Full outer join projects with departments
proj_dept_full = projects_df.join(
    departments_df,
    projects_df["department"] == departments_df["dept_name"],
    "full"
)

print("\nFull outer join - All projects and all departments:")
proj_dept_full.select(
    "project_name", "department", "dept_name", "description"
).orderBy(
    coalesce(col("department"), col("dept_name")).asc()
).show()

### Full Outer Join Analysis

Key observations:
- **All data preserved** (6 employees + 5 departments - 3 matches = 8 rows)
- **Frank** (Finance) appears with NULL department info
- **Marketing & Finance** departments appear with NULL employee info
- **Complete reconciliation** of both datasets

**Full outer joins give you the complete picture of data relationships.**

## 🔗 Multiple Table Joins

### Joining More Than Two DataFrames

In [ ]:
# Multiple table joins
print("🔗 MULTIPLE TABLE JOINS")
print("=" * 50)

# Method 1: Sequential joins
emp_dept = employees_df.join(
    departments_df,
    employees_df["department"] == departments_df["dept_name"],
    "inner"
)

emp_dept_proj = emp_dept.join(
    projects_df,
    emp_dept["department"] == projects_df["department"],
    "inner"
)

print("Three-way join (Employees → Departments → Projects):")
emp_dept_proj.select(
    "name", "department", "project_name", "team_size", "location"
).show()

# Method 2: Chain joins with different conditions
comprehensive_join = employees_df.join(
    departments_df,
    employees_df["department"] == departments_df["dept_name"],
    "left"
).join(
    projects_df,
    employees_df["department"] == projects_df["department"],
    "left"
)

print("\nComprehensive left join (all employees, matching dept/project):")
comprehensive_join.select(
    "name", "department", "project_name", "description", "location"
).orderBy("name").show()

# Method 3: Using SQL-style joins
employees_df.createOrReplaceTempView("employees")
departments_df.createOrReplaceTempView("departments")
projects_df.createOrReplaceTempView("projects")

sql_join = spark.sql("""
    SELECT e.name, e.department, p.project_name, d.location,
           e.salary, p.team_size
    FROM employees e
    INNER JOIN departments d ON e.department = d.dept_name
    LEFT JOIN projects p ON e.department = p.department
    ORDER BY e.salary DESC
""")

print("\nSQL-style multi-table join:")
sql_join.show()

# Performance comparison
print(f"\nResult counts:")
print(f"- Sequential joins: {emp_dept_proj.count()} rows")
print(f"- Comprehensive join: {comprehensive_join.count()} rows")
print(f"- SQL join: {sql_join.count()} rows")

## 🎛️ Advanced Join Conditions

### Non-Equality and Complex Join Conditions

In [ ]:
# Advanced join conditions
print("🎛️ ADVANCED JOIN CONDITIONS")
print("=" * 50)

# Create salary ranges for demonstration
salary_ranges = [
    ("Entry", 0, 60000),
    ("Mid", 60000, 80000),
    ("Senior", 80000, 100000),
    ("Executive", 100000, 999999)
]

ranges_df = spark.createDataFrame(salary_ranges, 
    ["level", "min_salary", "max_salary"])

print("Salary ranges:")
ranges_df.show()

# Non-equality join (salary within range)
salary_join = employees_df.join(
    ranges_df,
    (employees_df["salary"] >= ranges_df["min_salary"]) & 
    (employees_df["salary"] < ranges_df["max_salary"]),
    "inner"
)

print("\nNon-equality join (salary ranges):")
salary_join.select(
    "name", "salary", "level", "min_salary", "max_salary"
).orderBy("salary").show()

# Self-join example (comparing salaries)
self_join = employees_df.alias("e1").join(
    employees_df.alias("e2"),
    (col("e1.department") == col("e2.department")) & 
    (col("e1.emp_id") != col("e2.emp_id")) &
    (col("e1.salary") < col("e2.salary"))
)

print("\nSelf-join (lower earners vs higher earners in same dept):")
self_join.select(
    col("e1.name").alias("lower_earner"),
    col("e1.salary").alias("lower_salary"),
    col("e2.name").alias("higher_earner"),
    col("e2.salary").alias("higher_salary"),
    col("e1.department")
).orderBy("department", "lower_salary").show()

# Complex multi-condition join
complex_join = employees_df.join(
    projects_df,
    (employees_df["department"] == projects_df["department"]) & 
    (employees_df["salary"] > projects_df["team_size"] * 10000),  # Salary > team_size * 10k
    "inner"
)

print("\nComplex condition join:")
complex_join.select(
    "name", "department", "salary", "project_name", "team_size",
    (col("team_size") * 10000).alias("threshold")
).show()

## ⚡ Join Performance Optimization

### Join Strategies and Best Practices

In [ ]:
# Performance optimization
print("⚡ JOIN PERFORMANCE OPTIMIZATION")
print("=" * 50)

# Create larger datasets for performance testing
large_employees = [
    (i, f"Employee_{i}", f"Dept_{(i%5)+1}", 50000 + (i % 50000)) 
    for i in range(10000)
]

large_departments = [
    (f"Dept_{i}", f"Department {i}", f"City_{i%10}") 
    for i in range(1, 6)
]

large_emp_df = spark.createDataFrame(large_employees, 
    ["emp_id", "name", "department", "salary"])
large_dept_df = spark.createDataFrame(large_departments, 
    ["dept_name", "description", "location"])

print(f"Large employee dataset: {large_emp_df.count():,} rows")
print(f"Department dataset: {large_dept_df.count()} rows")

# Test different join configurations
import time

# Configuration 1: Default join
start_time = time.time()
default_join = large_emp_df.join(large_dept_df, 
    large_emp_df["department"] == large_dept_df["dept_name"], "inner")
default_count = default_join.count()
default_time = time.time() - start_time

print(f"\nDefault join: {default_count:,} rows in {default_time:.3f} seconds")

# Configuration 2: Broadcast join (force small table broadcast)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "1MB")  # Very small threshold

start_time = time.time()
broadcast_join = large_emp_df.join(
    F.broadcast(large_dept_df), 
    large_emp_df["department"] == large_dept_df["dept_name"], 
    "inner"
)
broadcast_count = broadcast_join.count()
broadcast_time = time.time() - start_time

print(f"Broadcast join: {broadcast_count:,} rows in {broadcast_time:.3f} seconds")

# Reset configuration
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "10MB")

print(f"\nPerformance comparison:")
print(f"- Default: {default_time:.3f}s")
print(f"- Broadcast: {broadcast_time:.3f}s")
if default_time > 0:
    improvement = default_time / broadcast_time
    print(f"- Broadcast is {improvement:.1f}x faster")

# Best practices summary
print("\n🚀 JOIN BEST PRACTICES:")
print("1. Use broadcast joins for small lookup tables")
print("2. Filter data before joining")
print("3. Choose appropriate join types")
print("4. Partition data on join keys when possible")
print("5. Cache frequently joined tables")
print("6. Monitor shuffle operations in Spark UI")

## 🚨 Common Join Mistakes

In [ ]:
# Common mistakes
print("🚨 COMMON JOIN MISTAKES")
print("=" * 50)

# Mistake 1: Cartesian explosion
print("❌ Mistake: Missing join condition (Cartesian product)")
try:
    bad_join = employees_df.join(departments_df)  # No join condition!
    print(f"Cartesian join result: {bad_join.count()} rows")
    print("(This creates every possible combination!)")
except Exception as e:
    print(f"Error: {str(e)[:100]}...")

print("\n✅ Solution: Always specify join condition")
good_join = employees_df.join(departments_df, 
    employees_df["department"] == departments_df["dept_name"])
print(f"Proper join result: {good_join.count()} rows")

# Mistake 2: Column name ambiguity
print("\n❌ Mistake: Ambiguous column names")
ambiguous_join = employees_df.join(
    projects_df, 
    employees_df["department"] == projects_df["department"]
)
print("Ambiguous columns after join:")
print(f"Columns: {[col for col in ambiguous_join.columns if 'department' in col]}")

print("\n✅ Solution: Use aliases or select specific columns")
clear_join = employees_df.alias("emp").join(
    projects_df.alias("proj"), 
    col("emp.department") == col("proj.department")
).select(
    col("emp.name"),
    col("emp.department").alias("emp_dept"),
    col("proj.project_name"),
    col("proj.department").alias("proj_dept")
)
print("Clear column names:")
clear_join.show(3)

# Mistake 3: Incorrect join type
print("\n❌ Mistake: Using inner join when left join is needed")
wrong_join = employees_df.join(departments_df, 
    employees_df["department"] == departments_df["dept_name"], "inner")
print(f"Inner join: {wrong_join.count()} rows (missing Frank)")

print("\n✅ Solution: Use left join to preserve all employees")
correct_join = employees_df.join(departments_df, 
    employees_df["department"] == departments_df["dept_name"], "left")
print(f"Left join: {correct_join.count()} rows (all employees included)")

# Mistake 4: Data skew causing performance issues
print("\n❌ Mistake: Ignoring data skew in joins")
print("Large partitions can cause slow, uneven task execution")

print("\n✅ Solutions for data skew:")
print("- Use salting: Add random suffix to join keys")
print("- Repartition data before joining")
print("- Use broadcast joins for small tables")
print("- Filter data to reduce skew")

## 🎯 Key Takeaways

### What You Learned:
- ✅ **`inner` join** - Only matching rows
- ✅ **`left` join** - All left rows + matching right rows
- ✅ **`right` join** - All right rows + matching left rows
- ✅ **`full` join** - All rows from both sides
- ✅ **Multiple table joins** - Joining 3+ DataFrames
- ✅ **Advanced join conditions** - Non-equality joins
- ✅ **Performance optimization** - Broadcast joins, filtering

### Join Performance Hierarchy:
- 🔸 **Broadcast Join**: Small table + large table (fastest)
- 🔸 **Shuffle Hash Join**: Medium tables with good key distribution
- 🔸 **Sort-Merge Join**: Large sorted tables
- 🔸 **Nested Loop Join**: Fallback, usually slowest

### Best Practices:
- 🔸 **Choose join type** based on data preservation needs
- 🔸 **Filter before joining** to reduce data volume
- 🔸 **Broadcast small tables** explicitly
- 🔸 **Use aliases** to avoid column name conflicts
- 🔸 **Always specify join conditions** to avoid Cartesian products
- 🔸 **Monitor join performance** in Spark UI

### Common Patterns:
- 🔸 **Lookup joins**: `large_df.join(broadcast(small_df), "key")`
- 🔸 **Preservation joins**: `main_df.join(ref_df, "key", "left")`
- 🔸 **Reconciliation joins**: `df1.join(df2, "key", "full")`
- 🔸 **Multi-table**: `df1.join(df2, "key").join(df3, "key")`

---

## 🚀 Next Steps

Now that you master basic joins, you're ready for:

1. **Broadcast Join Optimization** - Advanced join performance
2. **Skew Join Handling** - Dealing with data skew
3. **Spark SQL Joins** - SQL syntax for joins
4. **Advanced Analytics** - Complex multi-table queries

**Joins are the foundation of relational data processing!**

---

**🎉 Congratulations! You now wield the power of DataFrame joins like a database expert!**